In [2]:
import geopandas as gpd
import gdal as gd
import matplotlib.pyplot as plt
import pandas as pd

## Inputs and Adjustments

In [12]:
# csv of the snow depth measurements 
depths = 'SnowDepthMeasurements_TVDW.csv'

## Import Data

In [31]:
# Read the files that Thomas Sent
df0 = gpd.read_file(depths)

# collected files from 20210211
df1 = pd.read_csv('LOG01.TXT')
df1.to_csv(r'2021_02_11_probe.csv', index = None)
df1 = gpd.read_file('2021_02_11_probe.csv')


# collect second log
df2 = pd.read_csv('LOG02.TXT')
df2.to_csv(r'2021_02_11_probe2.csv', index = None)
df2 = gpd.read_file('2021_02_11_probe2.csv')
df0

,Date,Time,Latitude,Longitude,Elevation,Depth,geometry
0,1/8/2021,1:41:00 PM,43.7396278,-116.1203842,5254,0.81,None
1,1/8/2021,1:42:00 PM,43.7396202,-116.1203613,5254,0.56,None
2,1/8/2021,1:43:00 PM,43.7396164,-116.1203461,5254,0.58,None
3,1/8/2021,1:44:00 PM,43.7396049,-116.1202927,5253,0.56,None
4,1/8/2021,1:45:00 PM,43.7396011,-116.1202698,5252,0.84,None
...,...,...,...,...,...,...,...
148,1/20/2021,2:56:15 PM,43.7356644,-116.1228409,5150,0.75,None
149,1/20/2021,2:57:03 PM,43.7357330,-116.1228714,5146,0.87,None
150,1/20/2021,2:58:50 PM,43.7356300,-116.1229095,5144,0.87,None
151,1/20/2021,2:59:47 PM,43.7354584,-116.1229935,5140,0.55,None


In [32]:
df1

,Count,DistanceAvg,Distance1,Distance2,Distance3,Distance4,Distance5,latitude,longitude,altitude,sat_count,HDOP,dateTime,startGPSTime,endGPSTime,status,geometry
0,0,1903,1903,1904,1900,1903,1894,43.763237,-116.1047058,5335,20,0.6,666371089,2021/02/11 15:04:47.500,2021/02/11 15:04:49.300,3,None
1,1,1897,2019,1982,1891,1897,1896,43.763237,-116.1047058,5335,19,0.7,666371097,2021/02/11 15:04:55.900,2021/02/11 15:04:57.600,3,None
2,2,2016,2016,2016,2017,1894,2017,43.7632332,-116.1047058,5336,20,0.6,666371105,2021/02/11 15:05:04.100,2021/02/11 15:05:05.800,3,None
3,3,1892,1900,1890,1895,1891,1892,43.7632332,-116.1047058,5336,21,0.6,666371122,2021/02/11 15:05:20.700,2021/02/11 15:05:22.300,3,None
4,4,903,912,884,910,886,903,43.7631416,-116.1044388,5342,16,0.7,666372369,2021/02/11 15:26:07.400,2021/02/11 15:26:09.100,3,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,110,0,4668,0,0,0,0,43.7546387,-116.0998001,6151,7,5.0,666375504,2021/02/11 16:18:23.800,2021/02/11 16:18:24.400,3,None
111,111,0,4990,4664,0,0,0,43.7546158,-116.0997772,6152,10,2.0,666375510,2021/02/11 16:18:29.500,2021/02/11 16:18:30.200,3,None
112,112,4676,4676,4683,4684,0,0,43.7545929,-116.0997543,6153,10,2.0,666375514,2021/02/11 16:18:32.600,2021/02/11 16:18:34.100,3,None
113,113,0,0,0,0,0,0,43.7545891,-116.0997391,6153,7,1.9,666375516,2021/02/11 16:18:36.100,2021/02/11 16:18:36.500,3,None


In [33]:
df2

,Count,DistanceAvg,Distance1,Distance2,Distance3,Distance4,Distance5,latitude,longitude,altitude,sat_count,HDOP,dateTime,startGPSTime,endGPSTime,status,geometry
0,0,0,0,0,0,0,0,43.7545395,-116.096283,6165,13,0.8,666376122,2021/02/11 16:28:42.300,2021/02/11 16:28:42.800,3,None
1,1,1875,1970,0,1873,1875,1892,43.7545242,-116.096283,6163,13,0.8,666376128,2021/02/11 16:28:46.700,2021/02/11 16:28:48.300,3,None
2,2,1991,1985,1989,1991,1991,1993,43.7545319,-116.0962982,6163,13,0.8,666376133,2021/02/11 16:28:52.900,2021/02/11 16:28:53.400,3,None
3,3,0,0,0,0,0,0,43.7545319,-116.0962982,6163,13,0.8,666376135,2021/02/11 16:28:55.100,2021/02/11 16:28:55.500,3,None
4,4,0,0,0,0,0,0,43.7545395,-116.0962982,6162,8,1.1,666376138,2021/02/11 16:28:57.500,2021/02/11 16:28:58.0,3,None
